In [453]:
import pandas as pd
import re 

##### 0. ITEMID

In [472]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ITEMS.csv")
df

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12482,14518,226757,GCSMotorApacheIIValue,GCSMotorApacheIIValue,metavision,chartevents,Scores - APACHE II,NaN,Text,NaN
12483,14519,226758,GCSVerbalApacheIIValue,GCSVerbalApacheIIValue,metavision,chartevents,Scores - APACHE II,NaN,Text,NaN
12484,14520,226759,HCO3ApacheIIValue,HCO3ApacheIIValue,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN
12485,14521,226760,HCO3Score,HCO3Score,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN


In [473]:
# print(list(df['CATEGORY'].unique()))

In [474]:
# print(list(df['LABEL'].unique()))

In [475]:
# 1. ARDS 논문의 12개 feature + ChatGPT에게 물어본 General한 LABEL
L_list = ['WBC', 'White Blood', 'Albumin', 'Platelet', 'pH', 'Lactate Level', 'Fi02', 'Pa02', 'Heart Rate', 'temperature', 'BMI', 'Body Mass Index',
          'PaCO2', 'Base Excess', 'Hemoglobin', 'Hematocrit', 'Sodium', 'Potassium', 'Chloride', 'Bicarbonate', 'Blood Urea Nitrogen', 'BUN', 'Creatinine', 
          'Glucose', 'Calcium', 'Magnesium', 'Phosphate', 'Bilirubin', 'Aspartate Aminotransferase', 'AST', 'Alanine Aminotransferase',
          'ALT','Alkaline', 'Phosphatase' ,'ALP','Protein']
# L_list = ['temperature c']
L_list = [s.lower().replace(' ', '') for s in L_list]

df['LABEL'] = df['LABEL'].str.lower() #소문자로 변경
df['LABEL'] = df['LABEL'].str.replace(' ', '', regex=True).str.strip()#공백 제거

pattern = '|'.join(L_list)
df1 = df[pd.notna(df['LABEL']) & df['LABEL'].str.contains(pattern)]
ID1 = df1["ITEMID"].tolist()
print("특정 LABEL인 ITEMID 개수: ", len(ID1))

# # 2. Category 기준 ITEMID 
# df['CATEGORY'] = df['CATEGORY'].str.lower() #소문자로 변경
df2 = df[df['CATEGORY']==""]
ID2 = df2["ITEMID"].tolist()
print("특정 Category인 ITEMID 개수", len(ID2))

# 3. ID1, ID2 합치기
ID_list = ID1+ID2
print("전체 ITEMID 개수: ", len(ID_list))

# 4. ID_list에 해당한는 data 확인
df = df[df["ITEMID"].isin(ID_list)]
df.drop(columns=["ROW_ID", "ABBREVIATION", "DBSOURCE", "LINKSTO"], inplace=True)
unique_labels = set(df["LABEL"].to_list())
# print(unique_labels)

특정 LABEL인 ITEMID 개수:  1063
특정 Category인 ITEMID 개수 0
전체 ITEMID 개수:  1063


/tmp/ipykernel_1751057/2410845090.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["ROW_ID", "ABBREVIATION", "DBSOURCE", "LINKSTO"], inplace=True)


##### 1. LABEVENTS

In [436]:
df1 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/LABEVENTS.csv")
df1 = df1[df1["ITEMID"].isin(ID_list)]
df1

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG


##### 2. OUTPUTEVENT

In [420]:
df2 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/OUTPUTEVENTS.csv")
df2 = df2[df2["ITEMID"].isin(ID_list)]
df2

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR


##### 3. MICROBIOLOGYEVENTS

In [421]:
df3 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/MICROBIOLOGYEVENTS.csv")
ID_list = [float(x) for x in ID_list]
df3 = df3[df3[["SPEC_ITEMID", "ORG_ITEMID", "AB_ITEMID"]].isin(ID_list).any(axis=1)]
df3

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION


##### 4. 문제?

In [459]:
print("Table 별 유니크한 ITEMID 개수")
df1 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/LABEVENTS.csv")
print("LABEVENTS table의 ITEMID 수: ", len(df1['ITEMID'].sort_values().unique()))
df2 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/OUTPUTEVENTS.csv")
print("OUTPUTEVENTS table의 ITEMID 수: ", len(df2['ITEMID'].sort_values().unique()))
df3 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/MICROBIOLOGYEVENTS.csv")
print("MICROBIOLOGYEVENTS table의 ITEMID 수: ", len(df3['SPEC_ITEMID'].sort_values().unique()))

Table 별 유니크한 ITEMID 개수
LABEVENTS table의 ITEMID 수:  726
OUTPUTEVENTS table의 ITEMID 수:  1155
MICROBIOLOGYEVENTS table의 ITEMID 수:  93


In [464]:
df1.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [465]:
df2.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN


In [466]:
df3.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
